In [ ]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
sys.path.append(root_path)

In [ ]:
from decimal import Decimal

# Market configuration
exchange = "binance_perpetual"
trading_pair = "WLD-USDT"
interval = "3m"

# Account configuration
initial_portfolio_usd = 1000
order_amount = Decimal("25")
n_levels = 1
leverage = 20
trade_cost = 0.0006

# Backtest period
start = "2023-01-01"
end = "2024-01-02"

# Triple barrier configuration
stop_loss = Decimal("0.015")
take_profit = Decimal("0.03")
time_limit = 60 * 60 * 12 # 12 hours
trailing_stop_activation_price_delta = Decimal("0.008")
trailing_stop_trailing_delta = Decimal("0.004")

In [ ]:
from hummingbot.smart_components.utils.order_level_builder import OrderLevelBuilder
from hummingbot.smart_components.strategy_frameworks.data_types import (
    TripleBarrierConf
)

# Building the order levels
order_level_builder = OrderLevelBuilder(n_levels=n_levels)
order_levels = order_level_builder.build_order_levels(
    amounts=order_amount,
    spreads=Decimal("0"),
    # for directional strategies we don't need spreads since we are going to use market orders to enter
    triple_barrier_confs=TripleBarrierConf(
        stop_loss=stop_loss, take_profit=take_profit, time_limit=time_limit,
        trailing_stop_activation_price_delta=trailing_stop_activation_price_delta,
        trailing_stop_trailing_delta=trailing_stop_trailing_delta),
)

In [ ]:
# Let's inpect the order levels
order_levels

In [ ]:
import sys
from hummingbot.data_feed.candles_feed.candles_factory import CandlesConfig
from quants_lab.controllers.supertrend import SuperTrend, SuperTrendConfig

# Controller configuration
length = 100
multiplier = 3.0
percentage_threshold = 0.01

# Creating the instance of the configuration and the controller
config = SuperTrendConfig(
    exchange=exchange,
    trading_pair=trading_pair,
    order_levels=order_levels,
    candles_config=[
        CandlesConfig(connector=exchange, trading_pair=trading_pair, interval=interval, max_records=sys.maxsize),
    ],
    leverage=leverage,
    length=length,
    multiplier=multiplier,
    percentage_threshold=percentage_threshold,
)
controller = SuperTrend(config=config)

In [ ]:
from quants_lab.strategy.strategy_analysis import StrategyAnalysis

from hummingbot.smart_components.strategy_frameworks.directional_trading.directional_trading_backtesting_engine import \
    DirectionalTradingBacktestingEngine

# Creating the backtesting engine and loading the historical data
engine = DirectionalTradingBacktestingEngine(controller=controller)
engine.load_controller_data("../../../data/candles")

In [ ]:
# Let's see what is inside the candles of the controller
engine.controller.candles

In [ ]:
engine.controller.candles[0].candles_df

In [ ]:
# Let's understand what is inside the processed data since this is what we are going to use when generating the signal ;)
engine.controller.get_processed_data()

In [ ]:
# Let's run the backtesting

backtesting_results = engine.run_backtesting(initial_portfolio_usd=initial_portfolio_usd,
                                             trade_cost=trade_cost,
                                             start=start, end=end)

In [ ]:
# Let's see what is inside the backtesting results
backtesting_results.keys()

In [ ]:
# now let's analyze each of the dataframes

# 1. The processed data: this is the data that we are going to use to generate the signal
backtesting_results["processed_data"]

In [ ]:
# 2. The executors dataframe: this is the dataframe that contains the information of the orders that were executed
backtesting_results["executors_df"]

In [ ]:
# 3. The results dataframe: this is the dataframe that contains the information of the pnl of the strategy
backtesting_results["results"]

In [ ]:
# Now let's analyze the results using the StrategyAnalysis class
strategy_analysis = StrategyAnalysis(
    positions=backtesting_results["executors_df"],
    candles_df=backtesting_results["processed_data"],
)

In [ ]:
# let's visualize the PNL over time of the strategy
strategy_analysis.pnl_over_time()

In [ ]:
strategy_analysis.create_base_figure(volume=False, positions=False, trade_pnl=True)
fig = strategy_analysis.figure()

In [ ]:
fig

In [ ]:
# Now let's see how we can add the SuperTrend to the plot

import plotly.graph_objects as go

super_trend_long = strategy_analysis.candles_df[strategy_analysis.candles_df[f"SUPERTd_{length}_{multiplier}"] == 1]
super_trend_short = strategy_analysis.candles_df[strategy_analysis.candles_df[f"SUPERTd_{length}_{multiplier}"] == -1]
# Add the SuperTrend line
fig.add_trace(go.Scatter(x=super_trend_long.index, y=super_trend_long[f'SUPERT_{length}_{multiplier}'],
                         mode='markers',
                         name='SuperTrend Long',
                         line=dict(color="green")),
              row=1, col=1)
# Add the SuperTrend line
fig.add_trace(go.Scatter(x=super_trend_short.index, y=super_trend_short[f'SUPERT_{length}_{multiplier}'],
                         mode='markers',
                         name='SuperTrend Short',
                         line=dict(color="red")),
              row=1, col=1)

fig

In [ ]:
# To see the trades we will need to select a lower timeframe due the restrictions and speed of the plotly library
start_time = "2023-11-03"
end_time = "2023-11-05"

processed_data_filtered = backtesting_results["processed_data"][
    (backtesting_results["processed_data"]["timestamp"] >= start_time) &
    (backtesting_results["processed_data"]["timestamp"] <= end_time)
]

executors_filtered = backtesting_results["executors_df"][
    (backtesting_results["executors_df"]["timestamp"] >= start_time) &
    (backtesting_results["executors_df"]["timestamp"] <= end_time)
]

In [ ]:
executors_filtered

In [ ]:

strategy_analysis = StrategyAnalysis(
    positions=executors_filtered,
    candles_df=processed_data_filtered,
)

strategy_analysis.create_base_figure(volume=False, positions=True, trade_pnl=True)
fig = strategy_analysis.figure()
super_trend_long = strategy_analysis.candles_df[strategy_analysis.candles_df[f"SUPERTd_{length}_{multiplier}"] == 1]
super_trend_short = strategy_analysis.candles_df[strategy_analysis.candles_df[f"SUPERTd_{length}_{multiplier}"] == -1]
# Add the SuperTrend line
fig.add_trace(go.Scatter(x=super_trend_long.index, y=super_trend_long[f'SUPERT_{length}_{multiplier}'],
                         mode='markers',
                         name='SuperTrend Long',
                         line=dict(color="green")),
              row=1, col=1)
# Add the SuperTrend line
fig.add_trace(go.Scatter(x=super_trend_short.index, y=super_trend_short[f'SUPERT_{length}_{multiplier}'],
                         mode='markers',
                         name='SuperTrend Short',
                         line=dict(color="red")),
              row=1, col=1)
fig

### Scatter of PNL per Trade
This bar chart illustrates the PNL for each individual trade. Positive PNLs are shown in green and negative PNLs in red, providing a clear view of profitable vs. unprofitable trades.


In [ ]:
import plotly.express as px

executors_df = backtesting_results["executors_df"]

fig = px.scatter(executors_df,  x="timestamp", y='net_pnl_quote', title='PNL per Trade',
             color='profitable', color_continuous_scale=['red', 'green'])
fig.show()

### Scatter Plot of Volume vs. PNL
This scatter plot explores the relationship between the trade volume and the PNL for each trade. It can reveal if larger volumes are associated with higher profits or losses.


In [ ]:
fig = px.scatter(executors_df, x='volume', y='net_pnl_quote', title='Trade Volume vs. PNL')
fig.show()

### Histogram of PNL Distribution
The histogram displays the distribution of PNL values across all trades. It helps in understanding the frequency and range of profit and loss outcomes.


In [ ]:
fig = px.histogram(executors_df, x='net_pnl_quote', title='PNL Distribution')
fig.show()


# Conclusion
We can see that the indicator has potential to bring good signals to trade and might be interesting to see how we can design a market maker that shifts the mid price based on this indicator.
A lot of the short signals are wrong but if we zoom in into the loss signals we can see that the losses are not that big and the wins are bigger and if we had implemented the trailing stop feature probably a lot of them are going to be profits.

# Next steps
- Filter only the loss signals and understand what you can do to prevent them
- Try different configuration values for the indicator
- Test in multiple markets, pick mature markets like BTC-USDT or ETH-USDT and also volatile markets like DOGE-USDT or SHIB-USDT